In [1]:
#source https://github.com/pytorch/examples/blob/master/mnist/main.py

from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

In [2]:
class Argument():
    def __init__(self, batch_size=64, test_batch_size=1000,epochs=14, lr=1.0,
                gamma=0.7,no_cuda=False, log_interval=100,save_model=False):
        
        self.batch_size = batch_size
        self.test_batch_size = test_batch_size
        self.epochs = epochs
        self.lr = lr
        self.gamma = gamma
        self.no_cuda = no_cuda
        self.log_interval = log_interval
        self.save_model = save_model


In [3]:


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


def main():
    args = Argument()
    use_cuda = not args.no_cuda and torch.cuda.is_available()

    device = torch.device("cuda" if use_cuda else "cpu")

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=args.batch_size, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=args.test_batch_size, shuffle=True, **kwargs)

    model = Net().to(device)
    optimizer = optim.Adadelta(model.parameters(), lr=args.lr)

    scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)
    for epoch in range(1, args.epochs + 1):
        train(args, model, device, train_loader, optimizer, epoch)
        test(args, model, device, test_loader)
        scheduler.step()

    if args.save_model:
        torch.save(model.state_dict(), "mnist_cnn.pt")


In [4]:
%%timeit -n 1 -r 2
main()

/home/mtech/2021/dibyo/.anaconda/envs/CPU2/lib/python3.12/site-packages/torch/nn/functional.py:1374: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.307250
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.496940
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.389667
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.247868
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.136164
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.022412
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.068313
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.192952
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.151380
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.076537

Test set: Average loss: 0.0538, Accuracy: 9832/10000 (98%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.069932
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.111665
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.079174
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.178983
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.018600
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.120927
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.066531
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.113297
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.088573
T

KeyboardInterrupt: 

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize the images to [0, 1] range
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Reshape the data to fit the model
x_train = x_train.reshape(-1, 28 * 28)
x_test = x_test.reshape(-1, 28 * 28)

# Build the model
model = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(784,)),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=5, batch_size=32, validation_split=0.2)

# Evaluate the model on test data
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f'\nTest accuracy: {test_acc:.4f}')

# Make predictions
predictions = model.predict(x_test)

# Example: Print the first prediction and its true label
import numpy as np

predicted_label = np.argmax(predictions[0])
true_label = y_test[0]
print(f'Predicted label: {predicted_label}, True label: {true_label}')

2024-10-06 19:25:41.002103: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-06 19:25:41.077538: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-06 19:25:41.101731: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-06 19:25:41.162886: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-06 19:25:43.800880: W ten

Epoch 1/5


I0000 00:00:1728222961.744859 1676803 service.cc:146] XLA service 0x7f8a940097f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1728222961.744887 1676803 service.cc:154]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
I0000 00:00:1728222961.744891 1676803 service.cc:154]   StreamExecutor device (1): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
I0000 00:00:1728222961.744894 1676803 service.cc:154]   StreamExecutor device (2): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
I0000 00:00:1728222961.744897 1676803 service.cc:154]   StreamExecutor device (3): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
I0000 00:00:1728222961.744900 1676803 service.cc:154]   StreamExecutor device (4): Tesla V100-PCIE-32GB, Compute Capability 7.0
I0000 00:00:1728222961.744903 1676803 service.cc:154]   StreamExecutor device (5): Tesla V100-PCIE-32GB, Compute Capability 7.0
I0000 00:00:1728222961.744906 1676803 service.cc:154]   Str

   2/1500 ━━━━━━━━━━━━━━━━━━━━ 1:42 68ms/step - accuracy: 0.1797 - loss: 2.406880

I0000 00:00:1728222969.712632 1676803 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1500/1500 ━━━━━━━━━━━━━━━━━━━━ 75s 43ms/step - accuracy: 0.8176 - loss: 0.5986 - val_accuracy: 0.9555 - val_loss: 0.1497
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 59s 39ms/step - accuracy: 0.9445 - loss: 0.1862 - val_accuracy: 0.9664 - val_loss: 0.1152
Epoch 3/5
 347/1500 ━━━━━━━━━━━━━━━━━━━━ 37s 32ms/step - accuracy: 0.9547 - loss: 0.1494

KeyboardInterrupt: 

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Preprocess the data
x_train = x_train.reshape((60000, 28, 28, 1)).astype('float32') / 255
x_test = x_test.reshape((10000, 28, 28, 1)).astype('float32') / 255

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Build the model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=1, batch_size=64, validation_split=0.2)

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}')

# Save the model weights
model.save_weights('mnist_model.weights.h5')

print("Model weights saved to 'mnist_model_weights.h5'")

 85/750 ━━━━━━━━━━━━━━━━━━━━ 21s 33ms/step - accuracy: 0.5844 - loss: 1.4018

In [5]:
pip install Pillow

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
from PIL import Image

def load_model():
    # Recreate the model architecture
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    
    # Load the saved weights
    model.load_weights('mnist_model.weights.h5')
    return model

def preprocess_image(image_path):
    # Load the image
    img = Image.open(image_path).convert('L')  # Convert to grayscale
    
    # Resize to 28x28 pixels
    img = img.resize((28, 28))
    
    # Convert to numpy array
    img_array = np.array(img)
    
    # Normalize and reshape
    img_array = img_array.reshape((1, 28, 28, 1)).astype('float32') / 255
    
    return img_array

def predict_digit(model, image):
    # Make prediction
    prediction = model.predict(image)
    
    # Get the predicted digit
    predicted_digit = np.argmax(prediction)
    
    return predicted_digit, prediction[0]

def main():
    # Load the model
    model = load_model()
    # load all .png files from current directory
    import os
    image_paths = [f for f in os.listdir() if f.endswith('.png')]
    for image_path in image_paths:
        processed_image = preprocess_image(image_path)
        # Predict
        predicted_digit, probabilities = predict_digit(model, processed_image)
        print(f"Image: {image_path.split('.')[0]}")
        #print the image with seaborn
        import matplotlib.pyplot as plt
        import seaborn as sns
        sns.set(style='whitegrid')
        plt.figure(figsize=(6, 3))
        plt.subplot(1, 2, 1)
        plt.imshow(processed_image.reshape(28, 28), cmap='gray')
        plt.axis('off')
        plt.subplot(1, 2, 2)
        plt.bar(np.arange(10), probabilities)
        plt.xlabel('Digit')
        plt.ylabel('Probability')
        plt.xticks(np.arange(10))
        plt.tight_layout()
        plt.show()
        print(f"Predicted digit: {predicted_digit}")

if __name__ == "__main__":
    main()

2024-10-09 16:27:45.908313: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-09 16:27:45.926408: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-09 16:27:45.944306: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-09 16:27:45.949639: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-09 16:27:45.963883: I tensorflow/core/platform/cpu_feature_guar

: 